# monthly-holding-returns

This example demonstrates the monthly and yearly returns map and the holding period returns feature of pinkfish.

In [1]:
import datetime

import matplotlib.pyplot as plt
import pandas as pd

import pinkfish as pf

# Format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# Set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = '^GSPC'
capital = 10000
start = datetime.datetime(*pf.SP500_BEGIN)
end = datetime.datetime.now()

Timeseries

In [4]:
# Fetch timeseries, select, finalize.
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=True)
ts, start = pf.finalize_timeseries(ts, start, dropna=True, drop_columns=['open', 'high', 'low'])

# Create tradelog and daily balance objects.
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

Algorithm

In [5]:
pf.TradeLog.cash = capital

# Loop through timeseries.
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # Buy.
    if tlog.shares == 0:
        tlog.buy(date, row.close)
    # Sell.
    elif end_flag:
        tlog.sell(date, row.close)

    # Record daily balance.
    dbal.append(date, row.close)

Retrieve logs

In [6]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [7]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
0,1957-03-04,44.06,2023-12-21,4732.65,4688.59,1059621.32,226,1059621.32,LONG,^GSPC


In [8]:
dbal.tail()

,high,low,close,shares,cash,leverage,state
date,,,,,,,
2023-12-15,1066579.37,1066579.37,1066579.37,226,42.44,1.00,-
2023-12-18,1071409.01,1071409.01,1071409.01,226,42.44,1.00,-
2023-12-19,1077694.09,1077694.09,1077694.09,226,42.44,1.00,-
2023-12-20,1061869.56,1061869.56,1061869.56,226,42.44,1.00,-
2023-12-21,1069621.32,1069621.32,1069621.32,0,1069621.32,1.00,X


Compute stats

In [9]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [10]:
pf.summary(stats)

,strategy
annual_return_rate,7.25
max_closed_out_drawdown,-56.77
best_month,23.49
worst_month,-30.94
sharpe_ratio,0.52
sortino_ratio,0.66
monthly_std,4.29
annual_std,15.87


Show the monthly and yearly returns using the monthly_returns_map function.

In [11]:
returns = dbal['close']
pf.monthly_returns_map(returns['1990':])

Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
1990,-8.5,0.9,2.4,-2.7,9.2,-0.9,-0.5,-9.4,-5.1,-0.7,6.0,2.5,-8.2
1991,4.1,6.7,2.2,0.0,3.9,-4.8,4.5,2.0,-1.9,1.2,-4.4,11.2,26.3
1992,-2.0,1.0,-2.2,2.8,0.1,-1.7,3.9,-2.4,0.9,0.2,3.0,1.0,4.5
1993,0.7,1.0,1.9,-2.5,2.3,0.1,-0.5,3.4,-1.0,1.9,-1.3,1.0,7.1
1994,3.2,-3.0,-4.6,1.2,1.2,-2.7,3.1,3.8,-2.7,2.1,-3.9,1.2,-1.5
1995,2.4,3.6,2.7,2.8,3.6,2.1,3.2,-0.0,4.0,-0.5,4.1,1.7,34.1
1996,3.3,0.7,0.8,1.3,2.3,0.2,-4.6,1.9,5.4,2.6,7.3,-2.2,20.3
1997,6.1,0.6,-4.3,5.8,5.9,4.3,7.8,-5.7,5.3,-3.4,4.5,1.6,31.0
1998,1.0,7.0,5.0,0.9,-1.9,3.9,-1.2,-14.6,6.2,8.0,5.9,5.6,26.7
1999,4.1,-3.2,3.9,3.8,-2.5,5.4,-3.2,-0.6,-2.9,6.3,1.9,5.8,19.5


Show the holding period map.  This shows what your annualized return would have been, had you started this strategy at the start of a given year, as shown in the leftmost column, and held it for a certain number of years.

In [12]:
returns = dbal['close']
pf.holding_period_map(returns['1990':])